In [ ]:
%reload_ext autoreload
%autoreload 2
%autoreload now

import asyncio
import json
import logging
import os
import random

from dotenv import load_dotenv
from openai import AsyncOpenAI

from semaphore import set_semaphore
from utils_experiment import run_reorder_task
from utils_metrics import lcs_length, bubble_sort_distance, combined_normalized_loss
from utils_openai import call_gpt4

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

load_dotenv()

In [ ]:
set_semaphore("gpt-4", 100)

In [ ]:
client = AsyncOpenAI()

In [ ]:
comics = os.listdir("data/xkcd")

In [ ]:
len(comics)

In [ ]:
random.seed(42)
# comics_sample = random.sample(comics, 10)
comics_sample = comics

In [ ]:
model_name="gpt-4o"
# model_name="gpt-4o-mini"
# model_name="o1-mini"

tasks = [
    run_reorder_task(
        client=client,
        panels_path=f"data/xkcd/{comic}",
        model_name=model_name,
    ) for comic in comics_sample if f"data/xkcd/{comic}" not in panels_done
]
print(len(tasks), "tasks")

results = []
for coro in asyncio.as_completed(tasks):
    result = await coro
    results.append(result)

    if len(results) % 100 == 0:
        print(f"Processed {len(results)} objects.")
        json.dump(results, open(f"results-xkcd-{model_name}.json", "w"))

json.dump(results, open(f"results-xkcd-{model_name}.json", "w"))

In [ ]:
len(results), len(comics)

In [ ]:
len([result for result in results if result['failed']])

In [ ]:
metrics = [combined_normalized_loss(result["correct_order"], result["predicted_order"]) for result in results]
# metrics = [combined_normalized_loss(result["correct_order"], random.sample(result["correct_order"], len(result['correct_order']))) for result in results]

In [ ]:
avg_metric = sum(metrics) / len(metrics)

In [ ]:
avg_metric

In [ ]:
i = 8
results[i], metrics[i]

In [ ]:
response = await call_gpt4(
    client,
    "Explain what is in those pictures",
    imgs_paths=[
        "data/xkcd/13-Canyon.jpeg/panel_0.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_1.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_2.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_3.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_4.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_5.jpg",
        "data/xkcd/13-Canyon.jpeg/panel_6.jpg",
    ]
)

In [ ]:
response